<a id='top'></a>


# $\texttt{GiRaFFEfood}$: Initial data for $\texttt{GiRaFFE}$

## Alfv&eacute;n Wave

$$\label{top}$$

This module provides another initial data option for $\texttt{GiRaFFE}$, drawn from [this paper](https://arxiv.org/abs/1310.3274) . This is a flat-spacetime test with initial data 
\begin{align}
A_x &= 0 \\
A_y &= \left \{ \begin{array}{lll}\gamma_\mu x - 0.015 & \mbox{if} & x \leq -0.1/\gamma_\mu \\
1.15 \gamma_\mu x - 0.03g(x) & \mbox{if} & -0.1/\gamma_\mu \leq x \leq 0.1/\gamma_\mu \\ 
1.3 \gamma_\mu x - 0.015 & \mbox{if} & x \geq 0.1/\gamma_\mu \end{array} \right. , \\
 A_z = &\ y - \gamma_\mu (1-\mu)x ,
\end{align}
which generates the magnetic field in the wave frame,
\begin{align}
B'^{x'}(x') = &\ 1.0,\ B'^y(x') = 1.0, \\
B'^z(x') = &\ \left \{ \begin{array}{lll} 1.0 & \mbox{if} & x' \leq -0.1 \\
				1.0+0.15 f(x') & \mbox{if} & -0.1 \leq x' \leq 0.1 \\
				1.3 & \mbox{if} & x' \geq 0.1 \end{array} \right. ,
\end{align}
and the electric field in the wave frame, 
$$E'^{x'}(x') = -B'^z(0,x') \ \ , \ \ E'^y(x') = 0.0 \ \ , \ \ E'^z(x') = 1.0  .$$

These are converted to the grid frame by 
\begin{align}
  B^x(0,x) = &\ B'^{x'}(\gamma_\mu x) , \\
  B^y(0,x) = &\ \gamma_\mu [ B'^y(\gamma_\mu x) - \mu E'^z(\gamma_\mu x) ] , \\ 
  B^z(0,x) = &\ \gamma_\mu [ B'^z(\gamma_\mu x) + \mu E'^y(\gamma_\mu x) ] , 
\end{align}
and
\begin{align}
  E^x(0,x) = &\ E'^{x'}(\gamma_\mu x) , \\ 
  E^y(0,x) = &\ \gamma_\mu [ E'^y(\gamma_\mu x) + \mu B'^z(\gamma_\mu x) ] ,\\ 
  E^z(0,x) = &\ \gamma_\mu [ E'^z(\gamma_\mu x) - \mu B'^y(\gamma_\mu x) ],
\end{align}
and the velocity is given by $$\mathbf{v} = \frac{\mathbf{E} \times \mathbf{B}}{B^2}$$ in flat spacetime. Additionally, $f(x)=1+\sin (5\pi x)$, $-1<\mu<1$ is the wave speed relative to the grid frame and $\gamma_\mu = (1-\mu^2)^{-1/2}$, and $g(x) = \cos (5\pi \gamma_\mu x)/\pi$.

For the eventual purpose of testing convergence, any quantity $Q$ evolves as $Q(t,x) = Q(0,x-\mu t)$

See [previous NRPy+ tutorial module](Tutorial-GiRaFFEfood_HO.ipynb) for more general detail on how this is used.

#### Table of Contents:
1. [Steps 0-1:](#preliminaries) Preliminaries
1. [Step 2:](#step2) Set the vector $A_k$
1. [Step 3:](#step3) Set the vectors $B^i$ and $E^i$ for the velocity
1. [Step 4:](#step4) Calculate $v^i$
1. [Step 5:](#step6) NRPy+ Module Code Validation


<a id='preliminaries'></a>

### Steps 0-1: Preliminaries \[Back to [top](#top)\]

Here, we will import the NRPy+ core modules and set the reference metric to Cartesian, set commonly used NRPy+ parameters, and set C parameters that will be set from outside the code eventually generated from these expressions. We will also set up a parameter to determine what initial data is set up, although it won't do much yet.
$$\label{preliminaries}$$

In [1]:
# Step 0: Import the NRPy+ core modules and set the reference metric to Cartesian
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

import reference_metric as rfm
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Step 1a: Set commonly used parameters.
thismodule = "GiRaFFEfood_HO_1D"
# Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")


<a id='step2'></a>
### Set the vector $A_k$
The vector potential is given as
\begin{align}
A_x &= 0 \\
A_y &= \left \{ \begin{array}{lll}\gamma_\mu x - 0.015 & \mbox{if} & x \leq -0.1/\gamma_\mu \\
1.15 \gamma_\mu x - 0.03g(x) & \mbox{if} & -0.1/\gamma_\mu \leq x \leq 0.1/\gamma_\mu \\ 
1.3 \gamma_\mu x - 0.015 & \mbox{if} & x \geq 0.1/\gamma_\mu \end{array} \right. , \\
 A_z = &\ y - \gamma_\mu (1-\mu)x .
\end{align}
First, however, we must set $$\gamma_\mu = (1-\mu^2)^{-1/2}$$ and $$g(x) = \cos (5\pi \gamma_\mu x)/\pi$$.
$$\label{step2}$$

In [3]:
mu_AW,M_PI = par.Cparameters("REAL",thismodule,["mu_AW","M_PI"]) # The wave speed and pi in C
gammamu = (1-mu_AW**2)**(-1/2)

# We'll use reference_metric.py to define x
x = rfm.xxCart[0]

g_AW = sp.cos(5*M_PI*gammamu)/M_PI

initialize_param() minor warning: Did nothing; already initialized parameter GiRaFFEfood_HO_1D::mu_AW


Now, we can define the vector potential.